# Prepare notebook

In [28]:
from ast import literal_eval
from bs4 import BeautifulSoup
import json
import googlesearch
from googletrans import Translator
import pandas as pd
import urllib

# Load data set, clean, & scrape web for additional information

In [2]:
books = pd.read_csv('./goodreads_library_export.csv')

In [3]:
books = books[['ISBN13', 'Title', 'Author', 'My Rating','Publisher',
       'Number of Pages', 'Year Published', 'Original Publication Year',
       'Date Added', 'Bookshelves', 'Exclusive Shelf',
       'Private Notes']]

books = books.loc[~(books['Exclusive Shelf'] == 'to-read')]
del books['Exclusive Shelf']

In [4]:
books['Date Added'] = pd.to_datetime(books['Date Added'])

In [5]:
books['recommended'] = books.Bookshelves.fillna(False).astype(bool)
del books['Bookshelves']

In [6]:
books['ISBN13'] = books['ISBN13'].apply(lambda s: s.replace('=','').replace('"', '').strip())

In [8]:
translator = Translator()
scraped_data = {}

for index, row in books.fillna('').iterrows():
    scraped_data[index] = {'goodreads_link': None,
                           'goodreads_shelves': [],
                           'goodreads_description': None,
                           'goodreads_author_bio': None}
    
    
    # Pull the Goodreads webpage
    for result in googlesearch.search('Goodreads '+ row['Title']+' '+row['Author'], 
                                      start = 0, stop =10):
        if ('goodreads' in result) & ('book/show' in result):
            print(result)
            scraped_data[index]['goodreads_link'] = result
            try:
                http_query_result= urllib.request.urlopen(result)
            except urllib.error.HTTPError:
                print('Error reading http')
                break
            else: 
                soup = BeautifulSoup(http_query_result, 'html.parser')
                for shelf in soup.select("a[title*=shelved]"):
                    scraped_data[index]['goodreads_shelves'] += [(shelf.attrs["href"]).split('=')[1]]
                #if len(scraped_data[index]['goodreads_shelves']) == 0:
                    #scraped_data[index]['goodreads_shelves'] = literal_eval(soup.text.split('googletag.pubads().setTargeting("shelf", ')[1].split(');')[0])
                if len(scraped_data[index]['goodreads_shelves']) == 0:
                    print('  Missing shelves')

                try:
                    description = soup.find("div", {'id':"description"}).text
                    language = translator.detect(description).lang
                    if language != 'en':
                        description = translator.translate(description, dest='en', src = language).text
                    scraped_data[index]['goodreads_description'] = description        
                except:
                    print('  Missing description') 
                    
                    
                try:    
                    scraped_data[index]['goodreads_author_bio'] = soup.find("div", {'class':'bookAuthorProfile__about'}).text
                except:
                    print('  No author bio available')
                break

https://www.goodreads.com/book/show/108295.Gay_New_York
  No author bio available
https://www.goodreads.com/en/book/show/40265834
https://www.goodreads.com/book/show/41824023-over-the-top
https://www.goodreads.com/en/book/show/39999461
https://www.goodreads.com/book/show/44285661-hurricanes?sa=X&ved=2ahUKEwjDjpC8gI7mAhXyGDQIHTsKDhcQ9QF6BAgKEAI
  No author bio available
https://www.goodreads.com/book/show/15797326-zoobiquity
https://www.goodreads.com/book/show/44303730-me
https://www.goodreads.com/book/show/29779239-the-art-of-living
https://www.goodreads.com/book/show/43682552-how-we-fight-for-our-lives?sa=X&ved=2ahUKEwiA95rJgI7mAhX4GTQIHT4RDCwQ9QF6BAgKEAI
https://www.goodreads.com/book/show/40554115-the-gendered-brain
  No author bio available
https://www.goodreads.com/book/show/89369.The_Te_of_Piglet
https://www.goodreads.com/book/show/20256572-the-evil-hours
https://www.goodreads.com/book/show/8034188-where-good-ideas-come-from
https://www.goodreads.com/book/show/320877.When_Languag

https://www.goodreads.com/book/show/17331349-think-like-a-freak
https://www.goodreads.com/book/show/27774748-the-power-paradox?sa=X&ved=2ahUKEwi22Oneg47mAhXT6Z4KHeAaBJ8Q9QF6BAgKEAI
https://www.goodreads.com/book/show/40854717-gmorning-gnight
https://www.goodreads.com/book/show/34323010-dollars-and-sense?sa=X&ved=2ahUKEwi-tNHmg47mAhXNs54KHZbgD2IQ9QF6BAgLEAI
https://www.goodreads.com/book/show/23503955-chopsticks
  No author bio available
https://www.goodreads.com/book/show/20893477-how-we-got-to-now
https://www.goodreads.com/book/show/139069.Endurance
https://www.goodreads.com/book/show/23418.The_Architecture_of_Happiness
https://www.goodreads.com/book/show/2590136-barracoon
https://www.goodreads.com/book/show/28525282-this-chair-rocks?sa=X&ved=2ahUKEwjzgYv8g47mAhVG_J4KHbrLA8AQ9QF6BAgLEAI
https://www.goodreads.com/book/show/38628.A_History_of_Illuminated_Manuscripts
https://www.goodreads.com/en/book/show/12989034
https://www.goodreads.com/book/show/40940539-love-life-and-elephants
https

  No author bio available
https://www.goodreads.com/book/show/210404.A_Sand_County_Almanac_and_Sketches_Here_and_There
https://www.goodreads.com/book/show/36609352-what-truth-sounds-like
https://www.goodreads.com/book/show/37877606-the-ravenmaster
https://www.goodreads.com/book/show/38740747-the-snow-leopard-project
https://www.goodreads.com/en/book/show/23743731
https://www.goodreads.com/book/show/18114095-joy-guilt-anger-love
https://www.goodreads.com/en/book/show/43782400
  No author bio available
https://www.goodreads.com/book/show/17876.Notes_from_Underground_White_Nights_The_Dream_of_a_Ridiculous_Man_and_Selections_from_The_House_of_the_Dead
https://www.goodreads.com/book/show/12502.McCarthy_s_Bar
  No author bio available
https://www.goodreads.com/book/show/493.My_Inventions
https://www.goodreads.com/book/show/40174034-naturally-tan
https://www.goodreads.com/book/show/39284186-no-friend-but-the-mountains
https://www.goodreads.com/book/show/43835469-the-ungrateful-refugee
https:/

https://www.goodreads.com/book/show/46722.The_Coming_Plague
  No author bio available
https://www.goodreads.com/book/show/22463.The_Origin_of_Species
https://www.goodreads.com/book/show/3836.Don_Quixote
https://www.goodreads.com/book/show/10073506-tinker-tailor-soldier-spy
https://www.goodreads.com/book/show/81779.The_Symposium
https://www.goodreads.com/book/show/71979.My_Point_And_I_Do_Have_One
https://www.goodreads.com/book/show/1362.The_Histories
https://www.goodreads.com/book/show/568236.A_Distant_Mirror
https://www.goodreads.com/book/show/261243.History_of_the_Peloponnesian_War
https://www.goodreads.com/book/show/79908.What_Am_I_Doing_Here_
https://www.goodreads.com/book/show/7042.The_Tale_of_Genji
https://www.goodreads.com/book/show/19089.Middlemarch
https://www.goodreads.com/book/show/593109.The_Saga_of_the_Volsungs
https://www.goodreads.com/book/show/27161964-gratitude
https://www.goodreads.com/book/show/25899336-when-breath-becomes-air
https://www.goodreads.com/book/show/68727

  No author bio available
https://www.goodreads.com/book/show/13202060-the-homing-instinct?sa=X&ved=2ahUKEwjM3oKDjY7mAhWCjp4KHRaKDuoQ9QF6BAgKEAI
https://www.goodreads.com/book/show/1762039.The_History_and_Power_of_Writing
https://www.goodreads.com/book/show/23644794-10-human
https://www.goodreads.com/book/show/9634191-feathers
https://www.goodreads.com/en/book/show/20312456
  No author bio available
https://www.goodreads.com/book/show/262579.The_Unfolding_of_Language
https://www.goodreads.com/book/show/25300818-battling-the-gods
https://www.goodreads.com/book/show/17822205-the-book
  No author bio available
https://www.goodreads.com/book/show/40745.Mindset
https://www.goodreads.com/book/show/15797799-on-the-path-to-enlightenment
https://www.goodreads.com/book/show/738032.Radiant_Mind
  No author bio available
https://www.goodreads.com/book/show/13167157-the-last-rhinos?sa=X&ved=2ahUKEwj03empjY7mAhVInJ4KHUtyABUQ9QF6BAgKEAI
https://www.goodreads.com/book/show/409377.Babylon_s_Ark
https:/

https://www.goodreads.com/book/show/25982738-the-mysterious-world-of-the-human-genome
https://www.goodreads.com/book/show/12414734-the-epigenetics-revolution
  No author bio available
https://www.goodreads.com/book/show/29430796-being-a-dog
https://www.goodreads.com/book/show/30231743-are-we-smart-enough-to-know-how-smart-animals-are?sa=X&ved=2ahUKEwi6wcemkI7mAhXGop4KHW9GBDEQ9QF6BAgLEAI
https://www.goodreads.com/book/show/17290708-cat-sense?sa=X&ved=2ahUKEwjRsLarkI7mAhXNvJ4KHQroBCkQ9QF6BAgKEAI
https://www.goodreads.com/book/show/22857252-a-buzz-in-the-meadow
https://www.goodreads.com/book/show/254705.Winter_World
https://www.goodreads.com/book/show/15791140-seeds-of-hope
https://www.goodreads.com/book/show/28256439-the-hidden-life-of-trees
https://www.goodreads.com/book/show/31522121-the-songs-of-trees
https://www.goodreads.com/book/show/1044031.The_Emerald_Planet
  No author bio available
https://www.goodreads.com/book/show/87040.Gathering_Moss?sa=X&ved=2ahUKEwj_reTFkI7mAhWKITQIHTi4Cx

In [9]:
books = books[[ 'Title', 'Author', 'Publisher', 'My Rating','Number of Pages', 'Year Published', 
                    'Original Publication Year', 'Date Added', 'recommended']].join(pd.DataFrame(scraped_data).T)

In [10]:
books.count()

Title                        610
Author                       610
Publisher                    594
My Rating                    610
Number of Pages              584
Year Published               597
Original Publication Year    571
Date Added                   610
recommended                  610
goodreads_link               610
goodreads_shelves            610
goodreads_description        605
goodreads_author_bio         440
dtype: int64

In [11]:
books.to_pickle('20191128_goodreads_book_data.pkl')

# Manually short shelves

In [12]:
shelves = pd.get_dummies(books.goodreads_shelves.apply(pd.Series).stack()).sum(level=0)
shelves = shelves.astype(bool)
shelves = shelves.reindex(books.index)

In [24]:
category_mappings = {'literature_fiction':[ 'arthurian',
                                             'british-literature',
                                             'classic-literature',
                                             'classics',
                                             'fairy-tales',
                                             'fantasy',
                                             'fiction',
                                             'folk-tales',
                                             'folklore',
                                             'historical-fiction',
                                             'italian-literature',
                                             'japanese-literature',
                                             'literary-fiction',
                                             'literature',
                                             'military-fiction',
                                             'mythology',
                                             'novels',
                                             'russian-literature',
                                             'science-fiction',
                                             'short-stories',
                                             'spanish-literature',
                                             'urban-fantasy'],
                    'history': ['10th-century',
                                '19th-century',
                                 'american',
                                 'american-history',
                                 'ancient-history',
                                 'art-history',
                                 'church-history',
                                 'civil-war',
                                 'european-history',
                                 'egyptology',
                                 'food-history',
                                 'gothic',
                                 'historical',
                                 'history',
                                 'history-of-science',
                                 'holocaust',
                                 'medieval',
                                 'medieval-history',
                                 'microhistory',
                                 'post-colonial',
                                 'prehistory',
                                 'world-history',
                                 'world-war-ii'],
                    'social_studies': [ 'activism',
                                         'african-american',
                                         'anthropology',
                                         'business',
                                         'communication',
                                         'crime',
                                         'cultural',
                                         'disability',
                                         'economics',
                                         'education',
                                         'entrepreneurship',
                                         'espionage',
                                         'feminism',
                                         'finance',
                                         'futurism',
                                         'gay',
                                         'gender',
                                         'gender-studies',
                                         'journalism',
                                         'language',
                                         'leadership',
                                         'lgbt',
                                         'linguistics',
                                         'management',
                                         'money',
                                         'parenting',
                                         'politics',
                                         'productivity',
                                         'queer',
                                         'race',
                                         'social-justice',
                                         'society',
                                         'sociology',
                                         'true-crime',
                                         'war',
                                         'witchcraft',
                                         'womens'],
                    'technology': [ 'artificial-intelligence',
                                     'computer-science',
                                     'computers',
                                     'hackers',
                                     'internet',
                                     'mathematics',
                                     'physics',
                                     'technology'],
                    'culture': [ 'adventure',
                                 'architecture',
                                 'art',
                                 'basketball',
                                 'books-about-books',
                                 'comedy',
                                 'cookbooks',
                                 'cooking',
                                 'crafts',
                                 'design',
                                 'drama',
                                 'essays',
                                 'fashion',
                                 'food',
                                 'gardening',
                                 'hip-hop',
                                 'horror',
                                 'humor',
                                 'jazz',
                                 'magic',
                                 'maps',
                                 'music',
                                 'mystery',
                                 'photography',
                                 'plays',
                                 'poetry',
                                 'sports',
                                 'tea',
                                 'theatre',
                                 'thriller',
                                 'tragedy',
                                 'travel',
                                 'writing'],
                    'natural_sciences': [ 'agriculture',
                                         'animals',
                                         'archaeology',
                                         'astronomy',
                                         'biology',
                                         'birds',
                                         'cartography',
                                         'cats',
                                         'chemistry',
                                         'conservation',
                                         'dinosaurs',
                                         'dogs',
                                         'earth-sciences',
                                         'ecology',
                                         'environment',
                                         'evolution',
                                         'geography',
                                         'geology',
                                         'natural-history',
                                         'nature',
                                         'palaeontology',
                                         'plants',
                                         'research',
                                         'science',
                                         'science-nature',
                                         'space',
                                         'wildlife'],
                    'philosophy': [ 'atheism',
                                     'buddhism',
                                     'christianity',
                                     'inspirational',
                                     'islam',
                                     'judaism',
                                     'personal-development',
                                     'philosophy',
                                     'religion',
                                     'self-help',
                                     'spirituality',
                                     'zen'],
                    'biography_memoir': [ 'autobiography',
                                         'biography',
                                         'biography-memoir',
                                         'memoir'],
                    'medicine':  ['brain',
                                 'disease',
                                 'fitness',
                                 'genetics',
                                 'health',
                                 'health-care',
                                 'medical',
                                 'medicine',
                                 'mental-health',
                                 'neuroscience',
                                 'nutrition',
                                 'psychology']
                    }

In [25]:
categories = pd.DataFrame(index = shelves.index)
for k,v in category_mappings.items():
    categories[k] = (shelves[v].sum(axis =1) > 0)

In [26]:
categories.sum()

literature_fiction     77
history               223
social_studies        163
technology             23
culture               169
natural_sciences      305
philosophy            128
biography_memoir      153
medicine              146
dtype: int64

In [29]:
shelves.to_pickle('20191128_goodreads_book_shelves.pkl')

categories.to_pickle('20191128_goodreads_book_categories.pkl')
json.dump(category_mappings, open("category_mappings.json","w"))